In [ ]:
import brilearn
import utilities
import metrics
import pandas as pd
import numpy as np

# Making a prediction

Data format for prediction should look like this:

In [ ]:
pd.read_excel('data_format.xlsx', index_col='Index', sheet_name='For prediction')

For example:

In [ ]:
X = pd.read_excel('prediction_data.xlsx', index_col='Index')
X.head()

Calling the pre-tuned model and predicting flow stress curve:

In [ ]:
# Load model
model = brilearn.Model()
model.load_model()

# Find Meyer coefficients (required for prediction)
if ('k' not in X.columns) and ('m' not in X.columns):
    X = utilities.meyer_coeffs(X)

# Make a prediction
Y = model.predict(X)

Columns of Y represent the strain vector (the x axis of the flow curve).
The dataframe values are the flow stress values computed for the given strain vector

In [ ]:
Y

# Fitting a model

Data format for fitting should look like this:

In [ ]:
pd.read_excel('data_format.xlsx', index_col='Index', sheet_name='For fitting')

Fitting a model on a given database, and evaluating the prediction of the model

In [ ]:
# Call train/validation/test databases
df_train = pd.read_excel(
    "fitting_data.xlsx", index_col="Index", sheet_name="Training"
)
df_val = pd.read_excel(
    "fitting_data.xlsx", index_col="Index", sheet_name="Validation"
)
df_test = pd.read_excel("fitting_data.xlsx", index_col="Index", sheet_name="Testing")

# X matrix consists of Meyer coefficients and material hardness
meyer_coeffs = ["k", "m", "Hardness"]
X_train = df_train[meyer_coeffs]
X_val = df_val[meyer_coeffs]
X_test = df_test[meyer_coeffs]

# Y matrix consists of Ludwig coefficients
ludwig_coeffs = ["Y", "K", "n"]
Y_train = df_train[ludwig_coeffs]
Y_val = df_val[ludwig_coeffs]
Y_test = df_test[ludwig_coeffs]

In [ ]:
# Call model
model = brilearn.Model()

# Define desired strains vector
strains = np.linspace(0.02, 0.15, 30)

# Fit model using data
model.fit(X_train, Y_train, strains, X_val, Y_val)

# Make a prediction
Y_pred = model.predict(X_test)

# Construct a flow stress curve from the Ludwig coefficients (needed to compare the prediction to the test case)
Y_test = utilities.construct_graphs(Y_test, Y_pred.columns)

# Get prediction errors
errors = metrics.get_errors(Y_pred, Y_test)
errors.head()